[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DSIMB/PoincareMSA/blob/master/PoincareMSA_colab.ipynb)

# Directory Parameters

In [62]:
# Update working directory

%cd /home/hugo/Bureau/PoincareMSA/

/home/hugo/Bureau/PoincareMSA


In [63]:
import sys

# Add the project root to Python path
project_root = "/home/hugo/Bureau/PoincareMSA"
if project_root not in sys.path:
    sys.path.append(project_root)

# Parameters

## Exemple : inférer et ajouter un nouveau point

Ce bloc montre comment utiliser la méthode `infer_embedding_for_point` de `PoincareEmbedding` pour estimer la position d'un nouveau point dans une projection déjà calculée.

Guide rapide :
- Si tu disposes des features du nouveau point (`new_feat`) ou d'un vecteur de distances `new_to_existing`, le code calcule une cible (distribution) puis infère l'embedding.
- Le vecteur estimé est retourné et ajoutable au CSV d'embed existant.

Exécute la cellule suivante (code) après avoir défini `path_embedding` et, si disponible, `features` et `new_feat` ou `new_to_existing`.

In [ ]:
# ======= Exemple d'utilisation de `infer_embedding_for_point` =======
# Ce code suppose que `path_embedding` est défini plus haut dans le notebook.
# Il essaie d'utiliser `features` + `new_feat` ou `new_to_existing` si disponibles.

import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import pairwise_distances
from scripts.build_poincare_map.model import PoincareEmbedding

# 1) Charger la projection existante
try:
    df = pd.read_csv(path_embedding)
except Exception as e:
    raise RuntimeError(f"Impossible de charger {path_embedding}: {e}")

if 'pm1' not in df.columns or 'pm2' not in df.columns:
    raise ValueError("Le fichier d'embeddings doit contenir les colonnes 'pm1' et 'pm2'.")

embs = df[['pm1', 'pm2']].to_numpy(dtype=float)
N, dim = embs.shape
print(f"Chargé {N} embeddings de dimension {dim}")

# 2) Construire un modèle et injecter les poids existants
model = PoincareEmbedding(size=N, dim=dim, gamma=gamma if 'gamma' in globals() else 2, lossfn='klSym', Qdist='laplace', cuda=False)
with torch.no_grad():
    model.lt.weight.data = torch.from_numpy(embs).float()

# 3) Préparer le vecteur `target` (taille N) : plusieurs options
if 'features' in globals() and 'new_feat' in globals():
    # calculer distances new_feat vs features
    d = pairwise_distances(np.asarray(new_feat).reshape(1, -1), np.asarray(features), metric=distance if 'distance' in globals() else 'cosine').flatten()
    target = np.exp(-d / (sigma if 'sigma' in globals() else 1.0))
    target = target / target.sum()
    print('Target calculée à partir de features (exp(-d/sigma))')

elif 'new_to_existing' in globals():
    arr = np.asarray(new_to_existing, dtype=float)
    if arr.shape[0] != N:
        raise ValueError('new_to_existing length must equal current number of points')
    target = np.exp(-arr / (sigma if 'sigma' in globals() else 1.0))
    target = target / target.sum()
    print('Target calculée à partir de new_to_existing')

else:
    # Démo : cible uniforme (si tu n'as rien d'autre)
    target = np.ones(N, dtype=float) / float(N)
    print("Aucun vecteur fourni : utilisation d'une cible uniforme (démo)")

# 4) Inférer l'embedding pour le nouveau point
new_emb = model.infer_embedding_for_point(target, n_steps=500, lr=0.05, init='random')
print('Embedding inféré :', new_emb)

# 5) Ajouter au DataFrame et sauvegarder
df_new = df.copy()
new_row = {'pm1': float(new_emb[0]), 'pm2': float(new_emb[1]), 'proteins_id': 'NEW_POINT_1'}
df_new = pd.concat([df_new, pd.DataFrame([new_row])], ignore_index=True)
out_path = path_embedding.replace('.csv', '_with_new.csv')
df_new.to_csv(out_path, index=False)
print(f'Projection augmentée sauvegardée dans {out_path}')

# Optionnel : visualiser rapidement le point ajouté (si la fonction existe)
try:
    from scripts.visualize_projection.pplots_new import plot_embedding
    plot_embedding(df_new, title=f'Projection avec point ajouté', color_col=None)
except Exception:
    print('Plot non réalisé (fonction de visualisation indisponible ou erreur)')